In [67]:
import bs4 as bs
from bs4 import BeautifulSoup
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import os
import numpy as np
import pandas as pd
import pandas_datareader as web
import pickle
import requests
import urllib
import csv
import quandl
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

plotly.tools.set_credentials_file(username='SuperJesse', api_key='pvdYUHMsTRjlRzqGYZE2')
plotly.offline.init_notebook_mode(connected=True)

In [34]:
def main():
    try:
        compile_data()
    except:
        main()

In [35]:
def compile_data():
    get_data_from_yahoo(reload_sp500=True)
    with open("sp500tickers.pickle","rb") as f:
        tickerList = pickle.load(f)
        
    main_df = pd.DataFrame()
    
    for count, ticker in enumerate(tickerList):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index("Date", inplace=True)
        
        df.rename(columns = {'Adj Close':ticker}, inplace=True)
        df.drop(['Open','High',"Low",'Close','Volume'], 1, inplace=True)
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')
            
        if count % 10 ==0:
            print(count)
        
    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')

In [36]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickerList = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickerList = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
        
    start = dt.datetime(2017,10,30)
    end = dt.datetime(2017,10,31)
    
    for ticker in tickerList:
        print('ticker')
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
#             df = web.DataReader(ticker, 'yahoo', start, end)
            df = web.DataReader(ticker, 'yahoo', end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

In [37]:
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'class':'wikitable sortable'})
    tickerList = []
    for row in table.findAll('tr')[1:10]: #Will only get 10 companies. Delete 10 for whole list.
        ticker=row.findAll('td')[0].text
        mapping = str.maketrans(".","-")
        ticker = ticker.translate(mapping)
        if not ("BHF" in row):
            tickerList.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickerList, f)
    
    print(tickerList)
    
    return tickerList

In [38]:
main()

c:\users\superjesse\appdata\local\programs\python\python35\lib\site-packages\bs4\__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\superjesse\appdata\local\programs\python\python35\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")




['MMM', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AMD', 'AAP']
ticker
Already have MMM
ticker
Already have ABT
ticker
Already have ABBV
ticker
Already have ACN
ticker
Already have ATVI
ticker
Already have AYI
ticker
Already have ADBE
ticker
Already have AMD
ticker
Already have AAP
0
                   MMM        ABT       ABBV         ACN       ATVI  \
Date                                                                  
2017-10-31  229.964996  54.205002  90.169998  142.274994  65.425003   

                   AYI        ADBE     AMD    AAP  
Date                                               
2017-10-31  167.490005  175.089996  10.985  81.82  


In [39]:
def reformatDataFrame(input):
    df = pd.read_csv(input)
    df.pop('Date')
    df = df.transpose()
    df.columns = ["Adj Close Price"] #,"Market Cap", "P/E Ratio", "Div Yield (%)", "52 Week Price Change (%)"]
    df.index.name = "Ticker"
    df2 = pd.DataFrame()
    for ticker in df.index:
#         site = "http://financials.morningstar.com/ajax/exportKR2CSV.html?t="+ticker
#         print(site)
#         fund = pd.read_csv(site, error_bad_lines=False, engine = 'python')
#         fundamentals = urllib.request.urlopen("http://financials.morningstar.com/ajax/exportKR2CSV.html?t="+ticker)
#         fundamentals = csv.reader(fundamentals)
#         print(fundamentals)

#         htmlText = requests.get("http://financials.morningstar.com/ajax/exportKR2CSV.html?t="+ticker).text
#         lines=BeautifulSoup(htmlText,"html.parser")
#         lines = lines.prettify()
#         fout = open("fund.csv", 'w')
#         for line in lines.split("\n"):
#             print(line)
#             fout.write(line + "\n")
#         fout.close()
        df2 = pd.read_csv("MMMKeyRatios.csv", index_col = 0)
        print(ticker)
        df2 = df2['TTM'].head(20)
#         print(df2.index)
#         print(df2[0])
#     tickerDF = pd.DataFrame(index = ticker, columns = [df2.index])
#     print(tickerDF)

    dict = {}
    for i in range(len(df2.index)):
        dict[str(df2.index[i])] = df2[i]
    df3 = pd.DataFrame(data = dict, index = df.index)
#     print(df3)
#     print(df)
    frames = [df,df3]
    df4 = pd.concat(frames, axis=1)
    return(df4)

In [65]:
CurrentStockData=reformatDataFrame('sp500_joined_closes.csv')
#look at plotly
#docker
#version control GIthub
print("/////////")
print(list(CurrentStockData))
print(CurrentStockData[0:1].tolist())


MMM
ABT
ABBV
ACN
ATVI
AYI
ADBE
AMD
AAP
/////////
['Adj Close Price', 'Book Value Per Share * USD', 'Cap Spending USD Mil', 'Dividends USD', 'Earnings Per Share USD', 'Free Cash Flow Per Share * USD', 'Free Cash Flow USD Mil', 'Gross Margin %', 'Net Income USD Mil', 'Operating Cash Flow USD Mil', 'Operating Income USD Mil', 'Operating Margin %', 'Payout Ratio % *', 'Revenue USD Mil', 'Shares Mil', 'Working Capital USD Mil']


AttributeError: 'DataFrame' object has no attribute 'tolist'

In [68]:
def vizualizeData1():
    #this is all experimenting with plotly
    CurrentStockData=reformatDataFrame('sp500_joined_closes.csv')
    
    trace1 = go.Bar(
    x=list(CurrentStockData),
    y=CurrentStockData[0],
    name='MMM'
    )
    trace2 = go.Bar(
        x=['giraffes', 'orangutans', 'monkeys'],
        y=[12, 18, 29],
        name='LA Zoo'
    )

    data = [trace1, trace2]
    layout = go.Layout(
        barmode='group'
    )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='Grouped-bar')
    
vizualizeData1()


MMM
ABT
ABBV
ACN
ATVI
AYI
ADBE
AMD
AAP


KeyError: 0

In [72]:
def vizualizeData2():
    #this is all experimenting with plotly AND PANDAS!!
    CurrentStockData=reformatDataFrame('sp500_joined_closes.csv')
    return CurrentStockData.iplot(kind='bar', filename='grouped-bar-chart2')
    
vizualizeData2()

MMM
ABT
ABBV
ACN
ATVI
AYI
ADBE
AMD
AAP


In [45]:
help(plotly.offline.plot)

Help on function plot in module plotly.offline.offline:

plot(figure_or_data, show_link=True, link_text='Export to plot.ly', validate=True, output_type='file', include_plotlyjs=True, filename='temp-plot.html', auto_open=True, image=None, image_filename='plot_image', image_width=800, image_height=600, config=None)
    Create a plotly graph locally as an HTML document or string.
    
    Example:
    ```
    from plotly.offline import plot
    import plotly.graph_objs as go
    
    plot([go.Scatter(x=[1, 2, 3], y=[3, 2, 6])], filename='my-graph.html')
    # We can also download an image of the plot by setting the image parameter
    # to the image format we want
    plot([go.Scatter(x=[1, 2, 3], y=[3, 2, 6])], filename='my-graph.html'
         image='jpeg')
    ```
    More examples below.
    
    figure_or_data -- a plotly.graph_objs.Figure or plotly.graph_objs.Data or
                      dict or list that describes a Plotly graph.
                      See https://plot.ly/python/ f